In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from exp.nb_09b import *
import time
from fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time

# Get imagenette data

In [3]:
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)

tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor]
bs=128

il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=4)

In [4]:
nfs = [32]*4

In [6]:
??AvgStatsCallback

# Rewrite Avg Stat callback for progress bar

In [11]:
??AvgStats

In [12]:
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
    
    def begin_fit(self):
        met_names = ['loss'] + [m.__name__ for m in self.train_stats.metrics]
        names = ['epoch'] + [f'train_{n}' for n in met_names] + [
            f'valid_{n}' for n in met_names] + ['time']
        self.logger(names)
    
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        self.start_time = time.time() # start timer for this epoch
        
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
    
    def after_epoch(self):
        stats = [str(self.epoch)] # self.epoch is like i, i.e. for i in range(epochs)
        for o in [self.train_stats, self.valid_stats]:           
            stats += [f'{v:.6f}' for v in o.avg_stats] 
        stats += [format_time(time.time() - self.start_time)]
        # [0,train_loss,train_metric1,train_metric2,val_loss,val_metric1,val_metric2, time for completing 1 epoch]
        # will be added to master bar (below)
        self.logger(stats)

# Progress bar callback

In [22]:
class ProgressCallback(Callback):
    _order=-1
    def begin_fit(self):
        # master_bar handles the count over the epochs
        # create one at the beginning
        self.mbar = master_bar(range(self.epochs)) 
        self.mbar.on_iter_begin()
        
        # changing the logger of the Learner to the write function of the master bar
        # so everything will be written in the master bar
        self.run.logger = partial(self.mbar.write, table=True)
        
    def begin_epoch(self): self.set_pb() # create progress bar
        
    def after_batch(self): self.pb.update(self.iter) # update progress bar after an iteration
    
    def begin_validate(self): self.set_pb() # create new progress bar for validation
    
    def after_fit(self): self.mbar.on_iter_end()
        
    def set_pb(self):
        # master_bar's child, progress_bar, is looping over all the batches, and disappear when done
        self.pb = progress_bar(self.dl, parent=self.mbar, auto_update=False)
        self.mbar.update(self.epoch)

In [23]:
cbfs = [partial(AvgStatsCallback,accuracy),
        CudaCallback,
        ProgressCallback,
        partial(BatchTransformXCallback, norm_imagenette)]

In [24]:
learn = get_learner(nfs, data, 0.4, conv_layer, cb_funcs=cbfs)

In [25]:
learn.fit(2)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.980774,0.299597,1.919026,0.334000,00:04
1,1.570598,0.463394,1.401038,0.514000,00:04
